In [6]:
import os
import pandas as pd

In [7]:
directory = "matrix/stats"
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

In [8]:
files[1]

'ARM_L1D32L1I64L21024_1000000_10.txt'

In [9]:
def get_config(filename):
    # Extract the configuration from the filename
    # Example filename: "arch_L1dL1iL2_4_8.txt"
    # Expected output: dict with arch, l1d, l1i, l2, num, dim
    if "No_Cache" in filename:
        arch, cache,_, num, dim = filename.split("_")
        dim = dim.split(".")[0]
        l1d = 0
        l1i = 0
        l2 = 0
    else:
        arch, cache, num, dim = filename.split("_")
        dim = dim.split(".")[0]

        l1d = cache.split("L")[1][2:]
        l1i = cache.split("L")[2][2:]
        l2 = cache.split("L2")[1]

    return {
        "arch": arch,
        "l1d": l1d,
        "l1i": l1i,
        "l2": l2,
        "num": num,
        "dim": dim,
    }


In [10]:
def get_results(filename):
    metrics = {
        "simSeconds",
        "simTicks",
        "simInsts",
        "simOps",
        "core.cpi",
        "core.ipc",
        "l1d-cache-0.demandHits::total",
        "l1d-cache-0.demandMisses::total",
        "l1i-cache-0.demandHits::total",
        "l1i-cache-0.demandMisses::total",
        "l2-cache-0.demandHits::total",
        "l2-cache-0.demandMisses::total"
    }

    results = {}
    with open(f"matrix/stats/{filename}", "r") as f:
        lines = f.readlines()
        for line in lines:
            for metric in metrics:
                if metric in line:
                    data = line.split()
                    if len(data) >= 2:
                        results[metric] = data[1]
                    break
    return results


In [11]:
merged = {}

In [12]:
df = pd.DataFrame([(get_config(files[0])| get_results(files[0]))])

In [13]:
for filename in files[1:]:
    print(filename)
    config = get_config(filename)
    results = get_results(filename)
    merged[filename] = config | results
    df = pd.concat([df, pd.DataFrame([merged[filename]])], ignore_index=True)

ARM_L1D32L1I64L21024_1000000_10.txt
ARM_L1D128L1I32L2256_1000000_80.txt
ARM_L1D32L1I64L21024_1000000_5.txt
ARM_L1D32L1I64L2256_1000000_40.txt
ARM_L1D32L1I64L2128_1000000_80.txt
x86_L1D32L1I64L2128_1000000_20.txt
ARM_L1D128L1I32L2128_1000000_40.txt
RISCV_L1D32L1I128L21024_1000000_5.txt
RISCV_L1D128L1I32L2128_1000000_10.txt
RISCV_L1D128L1I128L21024_1000000_5.txt
x86_L1D64L1I128L2128_1000000_5.txt
RISCV_L1D32L1I32L2256_1000000_5.txt
x86_L1D128L1I64L21024_1000000_10.txt
x86_L1D32L1I128L2128_1000000_40.txt
RISCV_L1D128L1I64L2256_1000000_20.txt
x86_L1D128L1I64L2512_1000000_10.txt
x86_L1D32L1I128L2512_1000000_5.txt
x86_L1D32L1I32L21024_1000000_20.txt
ARM_L1D32L1I32L2512_1000000_20.txt
x86_L1D64L1I128L2256_1000000_5.txt
ARM_L1D64L1I32L2256_1000000_80.txt
x86_L1D32L1I64L2512_1000000_40.txt
RISCV_L1D32L1I64L2256_1000000_40.txt
RISCV_L1D64L1I32L2256_1000000_40.txt
RISCV_L1D32L1I64L2512_1000000_10.txt
ARM_L1D64L1I64L21024_1000000_10.txt
ARM_L1D64L1I128L21024_1000000_40.txt
ARM_L1D32L1I64L2128_1000

In [14]:
df

,arch,l1d,l1i,l2,num,dim,simSeconds,simTicks,simInsts,simOps,l1d-cache-0.demandHits::total,l1d-cache-0.demandMisses::total,l1i-cache-0.demandHits::total,l1i-cache-0.demandMisses::total,l2-cache-0.demandHits::total,l2-cache-0.demandMisses::total,core.cpi,core.ipc
0,ARM,128,64,1024,1000000,20,0.002198,2197972000,1479303,1689401,649878,586,1478796,531,819,962,1.485793,0.673041
1,ARM,32,64,1024,1000000,10,0.000619,619080000,404565,462988,154143,553,404066,521,794,864,1.530157,0.653528
2,ARM,128,32,256,1000000,80,0.058561,58560726000,39188736,42408450,19240209,989,39188253,545,1686,1216,1.494323,0.669199
3,ARM,32,64,1024,1000000,5,0.000282,281722000,172171,192213,50024,550,171671,521,750,859,1.636102,0.611209
4,ARM,32,64,256,1000000,40,0.010422,10421619000,7021893,7837955,3299807,988,7021385,540,1061,1236,1.484154,0.673784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,x86,128,128,1024,1000000,5,0.000434,433545000,226849,414226,67185,272,298134,678,607,1004,1.910841,0.523330
551,x86,32,32,512,1000000,10,0.000879,878761000,490173,881503,171032,311,637413,680,664,1002,1.792559,0.557862
552,x86,64,32,128,1000000,40,0.013567,13566832000,7927896,12666523,3301021,583,10150738,689,626,1344,1.711246,0.584370
553,x86,32,128,128,1000000,10,0.000879,878725000,490173,881503,171032,311,637415,678,662,1002,1.792486,0.557884


In [16]:
df.to_csv("results.csv", index=False)